In [237]:
using Symbolics

In [238]:
@variables x y C₁ C₂ C₃;

In [239]:
Dx = Differential(x)
Dy = Differential(y);

In [240]:
ϕ = C₁ * x^2 + C₂ * x * y + C₃ * y^2

C₁*(x^2) + C₂*x*y + C₃*(y^2)

In [241]:
σₓ = expand_derivatives(Dx(Dx(ϕ)))

2C₁

In [242]:
using LowLevelFEM
import LowLevelFEM as FEM

gmsh.initialize()

In [243]:
gmsh.open("beam.geo")

Info    : Reading 'beam.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.00373052s, CPU 0.000736s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.119862s, CPU 0.111326s)
Info    : Meshing order 2 (curvilinear on)...
Info    : [  0%] Meshing curve 1 order 2
Info    : [ 30%] Meshing curve 2 order 2
Info    : [ 50%] Meshing curve 3 order 2
Info    : [ 70%] Meshing curve 4 order 2
Info    : [ 90%] Meshing surface 1 order 2
Info    : Surface mesh: worst distortion = 1 (0 elements in ]0, 0.2]); worst gamma = 0.87295
Info    : Done meshing order 2 (Wall 0.0245472s, CPU 0.022639s)
Info    : 5033 nodes 2630 elements
Info    : Done reading 'beam.geo'


In [244]:
#gmsh.fltk.run()

In [245]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type=:PlaneStress);

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 100)


In [246]:
M = 1
l = 100
h = 10
p = -1
load = FEM.load("top", fy=p)
supp = FEM.displacementConstraint("left", ux=0, uy=0);

In [247]:
q = FEM.solveDisplacement(problem, [load], [supp]);

In [248]:
S = FEM.solveStress(problem, q)
S = FEM.elementsToNodes(problem, S);

In [249]:
sx = FEM.showDoFResults(problem, S, :sx)
sy = FEM.showDoFResults(problem, S, :sy)
sxy = FEM.showDoFResults(problem, S, :sxy)

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


2

In [250]:
exactx(x, y, z) = p * (2y - h) * (10 * y^2 - 10 * h * y - 15 * x^2 + 30 * l * x - 15 * l^2 + h^2) / (5 * h^3)
exacty(x, y, z) = -p * (2y - 3h) * y^2 / h^3
exactxy(x, y, z) = 6p * (x - l) * (y - h) * y / h^3

exactxy (generic function with 1 method)

In [251]:
sx0 = FEM.displacementConstraint("body", ux=exactx, uy=exacty)
sxy0 = FEM.displacementConstraint("body", ux=exactxy)
dof = length(q)
ssx = zeros(dof)
ssxy = zeros(dof)
FEM.applyBoundaryConditions!(problem, ssx, [sx0])
FEM.applyBoundaryConditions!(problem, ssxy, [sxy0])

In [252]:
sx1 = FEM.showDoFResults(problem, ssx, :ux, name="sx-exact")
sy1 = FEM.showDoFResults(problem, ssx, :uy, name="sy-exact")
sxy1 = FEM.showDoFResults(problem, ssxy, :ux, name="sxy-exact")

5

In [253]:
gmsh.fltk.run()

XRequest.18: BadValue 0x0


In [254]:
gmsh.finalize()